In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.datasets import fetch_california_housing

In [2]:
data = fetch_california_housing(as_frame=True)
data.frame

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


In [3]:
df = data.frame

In [ ]:
# extract x and y as numpy array format.

In [4]:
x = df.iloc[:, :-1].to_numpy()
y = df.iloc[:, -1].to_numpy()

In [5]:
n_sample, n_feature = x.shape
print(n_sample)
print(n_feature)

20640
8


In [ ]:
# add constant to x.

In [6]:
x_c = np.c_[np.ones(n_sample), x]

In [12]:
# compute the regression coefficients using matrix multiplication.

In [7]:
xtx = np.matmul(x_c.T, x_c)
xtx_inv = np.linalg.inv(xtx)

beta = np.matmul(np.matmul(xtx_inv, x_c.T), y)

In [ ]:
# make predictions for the dataset.

In [8]:
y_hat = np.matmul(x_c, beta)

In [ ]:
# compute SST, SSR, and SSE

In [9]:
y_bar = np.mean(y)
sst = np.sum((y - y_bar) ** 2)
ssr = np.sum((y_hat - y_bar) ** 2)
sse = np.sum((y - y_hat) ** 2)

In [ ]:
# compute mse and msr

In [10]:
mse = sse / (n_sample - n_feature - 1)
msr = ssr / n_feature

In [ ]:
# compute f-statistic

In [11]:
f = msr / mse

In [ ]:
# compute standard error of regression coefficients.

In [12]:
se_beta = np.sqrt(np.diag(mse * xtx_inv))

In [ ]:
# compute t-statistics

In [13]:
t = beta / se_beta

In [ ]:
# compute decision boundary of t-statistic given significance level 5%.

In [14]:
alpha = 0.05
cv = stats.t.ppf(1 - 0.5 * alpha, n_sample - n_feature - 1)

In [ ]:
# test if t-statistics are larger than the boundary.

In [15]:
print(np.abs(t) > cv)

[ True  True  True  True  True False  True  True  True]


In [51]:
# do the test with p-value

In [16]:
p_val = (1 - stats.t.cdf(np.abs(t), n_sample - n_feature - 1)) * 2
print(np.round(p_val, decimals=3))

[0.    0.    0.    0.    0.    0.402 0.    0.    0.   ]


In [ ]:
# compute R2 and adjusted R2.

In [17]:
r2 = ssr / sst
adj_r2 = 1 - (sse / (n_sample - n_feature - 1)) / (sst / (n_sample - 1))

In [18]:
print(r2)
print(adj_r2)

0.6062326852022651
0.6060799956298182


In [56]:
# fit the linear regression model using sklearn library.

In [19]:
from sklearn.linear_model import LinearRegression

In [20]:
lin_reg = LinearRegression()
lin_reg.fit(x, y)

LinearRegression()

In [61]:
# compute the R2 and adjusted R2.

In [21]:
r2 = lin_reg.score(x, y)
print(r2)

0.606232685199805


In [22]:
adj_r2 = 1 - (((n_sample - 1) / (n_sample - n_feature - 1)) * (1 - r2))
print(adj_r2)

0.606079995629818


In [71]:
# compute the r2 for each variable in x as a target by using the other variables as explanatory variables.

In [23]:
r2_list = []
for i in range(n_feature):
    # create a linear regression object.
    reg_i = LinearRegression()
    # extract i-th column of x as a target.
    x_i = x[:, i]
    # extract other column indexes first.
    other_indexes = np.setdiff1d(np.arange(n_feature), i)
    # extract other columns of x as independent variables.
    x_other = x[:, other_indexes]
    # fit the linear regression model.
    reg_i.fit(x_other, x_i)
    # compute its R2 and save it into the list.
    r2_i = reg_i.score(x_other, x_i)
    r2_list.append(r2_i)

r2_list = np.array(r2_list)

In [24]:
np.round(r2_list, decimals=3)

array([0.6  , 0.194, 0.88 , 0.857, 0.121, 0.008, 0.892, 0.888])

In [ ]:
# compute vif for variables.

In [25]:
vif = 1 / (1 - r2_list)

In [26]:
vif

array([2.50129451, 1.24125412, 8.34278562, 6.99499477, 1.13812508,
       1.00832445, 9.29762437, 8.96226347])

In [ ]:
# compute performance metrics for a full model.

In [27]:
mse = np.mean((y - y_hat) ** 2)
mae = np.mean(np.abs(y - y_hat))

In [28]:
print(mse, mae)

0.524320986184607 0.5311643817520755
